In [1]:
import os
from trade.binanceExch import Client, enums
from trade.binanceExch.indicators.MA import MA
from functools import reduce
import pandas as pd

from dotenv import load_dotenv
load_dotenv('/home/jb/PycharmProjects/binanceTradeDj/binanceTrade/.env')


True

In [2]:
import time

"""
class SymbolsTFData
- при обращении к def get_all_indexpower и def get_indexpower проверять актуальность уже имеющихся данных
    и уменьшить количество обращений к серверу  Binance
-
"""
class SymbolsTFData:
    client = Client(test_mode=False)

    def __init__(self, interval: str, short=5, middle=13, long=34, limit=500):
        if interval not in enums.INTERVALS:
            raise Exception("Incorrect parameter <interval>")
        self._interval = interval

        if not all([isinstance(short, int), isinstance(middle, int), isinstance(long, int), isinstance(limit, int)]):
            raise Exception("Parameters <short>, <middle> and <long> must be int")
        self._short = short
        self._middle = middle
        self._long = long
        self._limit = limit

        # {symbol: {
        #     'time': time.time()*1000,
        #     'data': pd.DataFrame()
        # }}
        self._dataframes = {}
        self._idx_dataframes = {}

        ls = reduce(
            lambda x,y: x + y,
            enums.IDX_COIN.values()
        )
        smbls = list(set(ls))
        self._symbols = list(set(smbls))

        self._columns=['Open_time', 'ma_power_'+str(self._short), 'ma_power_'+str(self._middle), 'ma_power_'+str(self._long)]
        # self._columns=['Open time', 'ma_'+str(self._short), 'ma_'+str(self._middle), 'ma_'+str(self._long), 'ma_power_'+str(self._short), 'ma_power_'+str(self._middle), 'ma_power_'+str(self._long)]

    def get_all_indexpower(self):
        for idxname in enums.IDX_COIN.keys():
            idx = self.get_indexpower(idxname)
            self._idx_dataframes[idxname] = {'time': int(time.time())*1000, 'data': idx.get(idxname)}

        return self._idx_dataframes

    def get_indexpower(self, idxname: str):
        """
        Return dictionary {idxname: pandas.DataFrame}
        Желательно добавить защиту от пустых DataFrame при расчете индекса
        """
        if idxname not in enums.IDX_COIN:
            raise Exception('Incorrect parameter <idxname> of get_indexpower() function')

        symbols = enums.IDX_COIN.get(idxname)
        idxset = {}
        for symbol in symbols:
            idxset.update(self.get_symbolpower(symbol))

        coin = idxname[3:]
        idx = pd.DataFrame(columns=self._columns)
        idx = idx.set_index('Open_time')
        for symbol, df in idxset.items():
            df = df.set_index('Open_time')
            if symbol.startswith(coin):
                idx = idx.add(df, fill_value=0)
            else:
                df = df.div(-1)
                idx = idx.add(df, fill_value=0)

        idx = idx.div(len(symbols))
        idx = idx.reset_index()

        return  {idxname: idx}

    def get_symbolpower(self, symbol):
        if symbol not in self._symbols:
            raise Exception(f'No data for symbol {symbol}')

        if self._dataframes.get(symbol) is None:
            self._load_symbol_data(symbol)

        time_now = int(time.time())*1000
        time_df = self._dataframes.get(symbol).get('time')

        # if data in df oldest 3 minutes recall data from server
        if time_now > time_df + 3*60*1000:
            self._load_symbol_data(symbol)
            df = self._dataframes.get(symbol).get('data')
        else:
            df = self._dataframes.get(symbol).get('data')

        df = df[self._columns]
        print(symbol,'\n',df.tail(2))

        return {symbol: df}

    def _load_all_data(self, smbls=None):
        symbols = self._symbols if smbls is None else smbls
        for symbol in symbols:
            self._load_symbol_data(symbol)

    def _load_symbol_data(self, symbol):
        """
        For all in symbols
        Take data from Binance.
        Add Columns ma_ and ma_diff_
        Save all in dict _dataframes = {symbol: pd.DataFrame}
        """
        print(symbol)
        df = self._get_klines(symbol)

        self._add_columns_ma_power(df, self._short)
        self._add_columns_ma_power(df, self._middle)
        self._add_columns_ma_power(df, self._long)

        self._dataframes[symbol] = {'time': int(time.time())*1000, 'data': df}

    def _add_columns_ma_power(self, df, ma_period, prefix='ma_', column_price='Close'):
        # add MA column
        column_ma = prefix + str(ma_period)
        df.loc[0, column_ma] = df.loc[0, column_price]
        for i in range(1, len(df)):
            df.loc[i, column_ma] = MA.ema(df.loc[i, column_price], df.loc[i-1, column_ma], ma_period)

        # add MA_DIFF column
        column_power = prefix+'power_' + str(ma_period)
        df[column_power] = MA.ema_power(df[column_ma].shift(1), df[column_ma])

    def _get_klines(self, symbol, lim=None):
        """
        Load klines data from server for <symbol> and <self._interval>
        Converts object(string) data to numeric
        return pandas.DataFrame()
        """
        limit = self._limit if lim is None else lim

        columns = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'Qt_asset_vol',
                   'Num_of_trades', 'Base_vol', 'Qt_vol', 'Ignr']

        df = pd.DataFrame(self.client.klines(symbol, self._interval, limit=limit), columns=columns)

        df['Close'] = df['Close'].astype('float')
        df['Open'] = df['Open'].astype('float')
        df['High'] = df['High'].astype('float')
        df['Low'] = df['Low'].astype('float')
        df['Volume'] = df['Volume'].astype('float')
        df['Qt_asset_vol'] = df['Qt_asset_vol'].astype('float')
        df['Base_vol'] = df['Base_vol'].astype('float')
        df['Qt_vol'] = df['Qt_vol'].astype('float')
        time.sleep(.5)

        return df

In [3]:
idx = SymbolsTFData('1h')
# all_idx = idx.get_all_indexpower()
# for k, v in all_idx.items():
#     print(k, v.get('time'))
#     print(v.get('data').tail(2))

usd = idx.get_indexpower('idxusd')
print(usd['idxusd'].tail(3))


BTCUSDT
BTCUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658498400000    0.002745     0.002089     0.001188
499  1658502000000    0.000917     0.001395     0.000961
ETHUSDT
ETHUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658498400000   -0.000775     0.000787     0.001351
499  1658502000000   -0.000755     0.000571     0.001231
BNBUSDT
BNBUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658498400000    0.002242     0.001556     0.001164
499  1658502000000    0.001987     0.001545     0.001182
SOLUSDT
SOLUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658498400000   -0.002407    -0.000651     0.000092
499  1658502000000   -0.001994    -0.000724     0.000021
ADAUSDT
ADAUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658498400000   -0.000333     0.000385     0.000441
499  1658502000000   -0.000421     0.000244     0.000381
XRPUSDT
XRPUSDT 
          Open_time  ma_power_5  ma_po

In [6]:
bnb = idx.get_indexpower('idxbnb')
bnb['idxbnb'].tail(5)

TRXBNB 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000    0.000977     0.000267    -0.000113
499  1658466000000   -0.000253    -0.000158    -0.000260
BNBETH 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000    0.000116    -0.000117    -0.000340
499  1658466000000   -0.001701    -0.000862    -0.000624
BNBBTC 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000    0.001106     0.001068     0.000753
499  1658466000000   -0.000017     0.000590     0.000579
SOLBNB 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.002547    -0.001438    -0.000791
499  1658466000000   -0.000050    -0.000529    -0.000466
ADABNB 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.001406    -0.000973    -0.000761
499  1658466000000   -0.000226    -0.000530    -0.000597
XRPBNB 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.001658    

,Open_time,ma_power_5,ma_power_13,ma_power_34
495,1658451600000,0.000706,0.000314,0.000179
496,1658455200000,0.000546,0.000302,0.000182
497,1658458800000,0.000129,0.000157,0.000131
498,1658462400000,0.000846,0.000461,0.000254
499,1658466000000,0.000648,0.000431,0.000254


In [7]:
bnb = idx10.get_indexpower('idxusd')
bnb['idxusd'].tail(5)

BTCUSDT
BTCUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.000828    -0.000348    -0.000172
499  1658466000000    0.001288     0.000490     0.000153
ETHUSDT
ETHUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000    0.000111     0.000818     0.000922
499  1658466000000    0.002799     0.001874     0.001343
BNBUSDT
BNBUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000    0.000246     0.000705     0.000582
499  1658466000000    0.001423     0.001146     0.000766
SOLUSDT
SOLUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.002173    -0.000705    -0.000219
499  1658466000000    0.000967     0.000432     0.000209
ADAUSDT
ADAUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.001104    -0.000265    -0.000196
499  1658466000000    0.001082     0.000553     0.000127
XRPUSDT
XRPUSDT 
          Open_time  ma_power_5  ma_po

,Open_time,ma_power_5,ma_power_13,ma_power_34
495,1658451600000,0.000963,-0.000876,-0.000612
496,1658455200000,0.002588,0.00009,-0.00024
497,1658458800000,0.001871,0.000138,-0.000201
498,1658462400000,0.00089,-0.000036,-0.000251
499,1658466000000,-0.000972,-0.000704,-0.000507


In [8]:
bnb1 = idx10.get_indexpower('idxusd')
bnb1['idxusd'].tail(5)

BTCUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.000828    -0.000348    -0.000172
499  1658466000000    0.001288     0.000490     0.000153
ETHUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000    0.000111     0.000818     0.000922
499  1658466000000    0.002799     0.001874     0.001343
BNBUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000    0.000246     0.000705     0.000582
499  1658466000000    0.001423     0.001146     0.000766
SOLUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.002173    -0.000705    -0.000219
499  1658466000000    0.000967     0.000432     0.000209
ADAUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.001104    -0.000265    -0.000196
499  1658466000000    0.001082     0.000553     0.000127
XRPUSDT 
          Open_time  ma_power_5  ma_power_13  ma_power_34
498  1658462400000   -0.0012

,Open_time,ma_power_5,ma_power_13,ma_power_34
495,1658451600000,0.000963,-0.000876,-0.000612
496,1658455200000,0.002588,0.00009,-0.00024
497,1658458800000,0.001871,0.000138,-0.000201
498,1658462400000,0.00089,-0.000036,-0.000251
499,1658466000000,-0.000972,-0.000704,-0.000507


In [2]:
import time
time_now = int(time.time())*1000
time_df = time_now + 1*60*1000
while time_now < time_df:
    print(time_df, '<>',time_now)
    time_now = int(time.time()*1000)
    time.sleep(1)

1658470266000 <> 1658470206000
1658470266000 <> 1658470206670
1658470266000 <> 1658470207671
1658470266000 <> 1658470208672
1658470266000 <> 1658470209674
1658470266000 <> 1658470210676
1658470266000 <> 1658470211677
1658470266000 <> 1658470212678
1658470266000 <> 1658470213679
1658470266000 <> 1658470214681
1658470266000 <> 1658470215682
1658470266000 <> 1658470216684
1658470266000 <> 1658470217685
1658470266000 <> 1658470218687
1658470266000 <> 1658470219688
1658470266000 <> 1658470220690
1658470266000 <> 1658470221691
1658470266000 <> 1658470222691
1658470266000 <> 1658470223706
1658470266000 <> 1658470224707
1658470266000 <> 1658470225709
1658470266000 <> 1658470226711
1658470266000 <> 1658470227712
1658470266000 <> 1658470228713
1658470266000 <> 1658470229715
1658470266000 <> 1658470230716
1658470266000 <> 1658470231717
1658470266000 <> 1658470232718
1658470266000 <> 1658470233719
1658470266000 <> 1658470234725
1658470266000 <> 1658470235727
1658470266000 <> 1658470236728
16584702